In [111]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, LlamaTokenizer
import random
from tqdm import tqdm
import re, torch
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from models.modelings_alignable_llama import *
from utils.train_utils import *


#### GPT-J Alpaca 6B

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "nlpcloud/instruct-gpt-j-fp16",
    cache_dir=CACHE_DIR,
    torch_dtype=torch.bfloat16
)
_ = model.to("cuda")
tokenizer = AutoTokenizer.from_pretrained(
    "nlpcloud/instruct-gpt-j-fp16",
    cache_dir=CACHE_DIR
)

In [ ]:
prompt = """Reason True or False for the following statement.\n3 is equal to 1.\n"""
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
attention_mask = tokenizer(prompt, return_tensors="pt").attention_mask.to("cuda")
model.eval()
outputs = model(
    input_ids,
    attention_mask=attention_mask
)
pred_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
generated_tokens = tokenizer.decode(pred_labels[0])
generated_tokens

#### StableLM 7B

In [ ]:
# model = AutoModelForCausalLM.from_pretrained(
#     "../../stablelm_7b/",
#     cache_dir=CACHE_DIR,
#     torch_dtype=torch.bfloat16
# )
# _ = model.to("cuda")
# tokenizer = AutoTokenizer.from_pretrained(
#     "../../stablelm_7b/",
#     cache_dir=CACHE_DIR
# )

#### Loading Dolly-12B

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "databricks/dolly-v2-12b",
    cache_dir=CACHE_DIR,
    torch_dtype=torch.bfloat16
)
_ = model.to("cuda")
tokenizer = AutoTokenizer.from_pretrained(
    "databricks/dolly-v2-12b",
    cache_dir=CACHE_DIR
)

#### Loading Alpaca-13B

In [2]:
alignment_config = {
    'layer': 25,
    "token_range" : [0, 1]
}
model = AlignableLlamaForCausalLM.from_pretrained(
    "../../alpaca_13b/",
    alignment_config=alignment_config,
    torch_dtype=torch.bfloat16
)
_ = model.to("cuda")
tokenizer = LlamaTokenizer.from_pretrained(
    "chavinlo/alpaca-13b",
    cache_dir=CACHE_DIR
)

loading configuration file ../../alpaca_13b/config.json
Model config LlamaConfig {
  "_name_or_path": "chavinlo/alpaca-13b",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 0,
  "eos_token_id": 1,
  "hidden_act": "silu",
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 13824,
  "max_position_embeddings": 2048,
  "max_sequence_length": 2048,
  "model_type": "llama",
  "num_attention_heads": 40,
  "num_hidden_layers": 40,
  "pad_token_id": -1,
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.28.0.dev0",
  "use_cache": true,
  "vocab_size": 32001
}

loading weights file ../../alpaca_13b/pytorch_model.bin.index.json
Instantiating AlignableLlamaForCausalLM model under default dtype torch.bfloat16.
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": -1,
  "transformers_version": "4.28.0.dev0"
}



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing AlignableLlamaForCausalLM.

Some weights of AlignableLlamaForCausalLM were not initialized from the model checkpoint at ../../alpaca_13b/ and are newly initialized: ['model.intervention_boundaries', 'model.intervention_population', 'model.inverse_rotate_layer.lin_layer.parametrizations.weight.original', 'model.rotate_layer.parametrizations.weight.original', 'model.inverse_rotate_layer.lin_layer.parametrizations.weight.0.base', 'model.temperature', 'model.rotate_layer.parametrizations.weight.0.base']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
loading configuration file ../../alpaca_13b/generation_config.json
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.28.0.dev0"
}

loading file tokenizer.model from cache at ../.cache/models--chavinlo--alpaca-13b/snapsho

#### Loading Alpaca-7B

In [2]:
alignment_config = {
    'layer': 25,
    "token_range" : [0, 1]
}
model = AlignableLlamaForCausalLM.from_pretrained(
    "../../alpaca_7b_bf16/",
    alignment_config=alignment_config,
    torch_dtype=torch.bfloat16
)
_ = model.to("cuda")
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path="../../alpaca_7b/",
    cache_dir=CACHE_DIR
)

loading configuration file ../../alpaca_7b_bf16/config.json
Model config LlamaConfig {
  "_name_or_path": "../../alpaca_7b/",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 0,
  "eos_token_id": 1,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "max_sequence_length": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "pad_token_id": -1,
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.28.0.dev0",
  "use_cache": true,
  "vocab_size": 32001
}

loading weights file ../../alpaca_7b_bf16/pytorch_model.bin.index.json
Instantiating AlignableLlamaForCausalLM model under default dtype torch.bfloat16.
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": -1,
  "transformers_version": "4.28.0.dev0"
}



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing AlignableLlamaForCausalLM.

Some weights of AlignableLlamaForCausalLM were not initialized from the model checkpoint at ../../alpaca_7b_bf16/ and are newly initialized: ['model.inverse_rotate_layer.lin_layer.parametrizations.weight.0.base', 'model.temperature', 'model.rotate_layer.parametrizations.weight.original', 'model.inverse_rotate_layer.lin_layer.parametrizations.weight.original', 'model.rotate_layer.parametrizations.weight.0.base', 'model.intervention_population', 'model.intervention_boundaries']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
loading configuration file ../../alpaca_7b_bf16/generation_config.json
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": -1,
  "transformers_version": "4.28.0.dev0"
}

loading file tokenizer.model
loading file added_tokens.json
loading file special

In [123]:
alpaca_prompt_template = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
%s

### Input:
%s

### Response:
"""

alpaca_prompt_template_no_inputs = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
%s

### Response:
"""

In [ ]:
alpaca_instruction = """Please say yes only if Sam is heavier than John, otherwise no."""
prompt = alpaca_prompt_template % (alpaca_instruction, "Sam weigh 112 lbs, John weigh 163 lbs", "")

# alpaca_instruction = """Does Donald Trump use computer?"""
# prompt = alpaca_prompt_template_no_inputs % (alpaca_instruction)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
attention_mask = tokenizer(prompt, return_tensors="pt").attention_mask.to("cuda")
model.eval()
outputs = model(
    input_ids,
    attention_mask=attention_mask
)
pred_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
generated_tokens = tokenizer.decode(pred_labels[0])

afc_1 = tokenizer.convert_tokens_to_ids("Yes")
afc_2 = tokenizer.convert_tokens_to_ids("No")
afc_1_prob = outputs.logits[:, -1][0][afc_1]
afc_2_prob = outputs.logits[:, -1][0][afc_2]
if afc_1_prob > afc_2_prob:
    afc = "Yes"
else:
    afc = "No"
print(f"afc label = {afc} ({afc_1_prob}/{afc_2_prob}) ; pred label = {generated_tokens}")
    

#### Working task: Pricing Tag Game

In [207]:
raw_prealign = factual_sampler(
    tokenizer,
    5000,
    game="pricing_tag"
)
prealign_dataset = Dataset.from_dict(
    {
        "input_ids": raw_prealign[0], 
        "labels": raw_prealign[1],
    }
).with_format("torch")
prealign_dataloader = DataLoader(
    prealign_dataset, batch_size=8
)

In [201]:
total_count = 0
correct_count = 0
model.eval()
with torch.no_grad():
    for step, inputs in enumerate(tqdm(prealign_dataloader)):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to(model.device)

        # aligning forward!
        outputs = model(
            input_ids=inputs['input_ids'],
            labels=inputs['labels'],
        )

        actual_test_labels = inputs['labels'][:, -1]
        pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)

        correct_labels = (actual_test_labels==pred_test_labels)

        total_count += len(correct_labels)
        correct_count += correct_labels.sum().tolist()
current_acc = round(correct_count/total_count, 2)
print(f"[WARNING: THIS NEEDS TO BE GOOD!] prealign task accuracy: {current_acc}")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [01:23<00:00,  7.44it/s]

[WARNING: THIS NEEDS TO BE GOOD!] prealign task accuracy: 0.85


#### Working task: Pricing Tag Game with Material Type

In [ ]:
examples = []
for i in range(100, 1000):
    examples += [{
        'a' : f"gold {str(i)[0]}.{str(i)[1]}{str(i)[2]} dollars",
        'output' : ['Yes', 'yes'] if i < 700 and i > 200 else ['No', 'no']
        # 'output' : ['No', 'no']
    }]

In [ ]:
# instruction = """Please say yes only if wood costs between 3.00 and 4.99 dollars, otherwise no."""
# instruction = """Please say yes only if metal costs between 2.00 and 6.99 dollars, otherwise no."""
alpaca_prompt_template = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
%s

### Input:
%s

### Response:
"""

In [ ]:
n_correct = 0
model.eval()
pred_labels = []
with torch.no_grad():
    for example in tqdm(examples):
        inputs = example['a']
        alpaca_prompt = alpaca_prompt_template % (instruction, inputs)

        input_ids = tokenizer(alpaca_prompt, return_tensors="pt").input_ids.to("cuda")
        generated_ids = model.generate(
            input_ids, do_sample=True, 
            temperature=0.01, max_length=input_ids[0].shape[0]+3,
        )
        generated_tokens = tokenizer.decode(generated_ids[0])
        pred_label = generated_tokens.split("</s>")[0].strip(".\n").split("\n")[-1].strip()
        correct_label = example['output']
        if pred_label in correct_label:
            n_correct += 1
        print(pred_label, correct_label, inputs)
        pred_labels += [pred_label]
        

In [ ]:
n_correct/len(examples)

In [ ]:
plot_data_x = []
plot_data_y = []
for i in range(len(pred_labels)):
    plot_data_x += [round((i+100)/100.0, 2)]
    plot_data_y += [0 if pred_labels[i] in ['No', 'no'] else 1]

In [ ]:
plt.figure(figsize=(10,1))
plt.plot(plot_data_x, plot_data_y, '-')
plt.show()